In [1079]:
covariance_matrix = [[1, 2, 3, 4],
                     [4, 3, 2, 1],
                     [90, -90, 50, -50],
                     [-100, 500, -90, 45]]

#covariance_matrix = [[11.0, 25.0, -30.0],
#                     [-3.0, 2.0, 50.0],
#                     [-10.0, 100.0, 50.0]]

def convert_matrix_to_decimals(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            matrix[i][j] =  Decimal(matrix[i][j])
    return matrix
                     

n = len(covariance_matrix)
import math  #for sqrt function
from decimal import Decimal  # to be more precise in arithmetics instead of floats
import copy

covariance_matrix = convert_matrix_to_decimals(covariance_matrix)

In [1080]:
# Task 1
# LU decomposition using Gauss Elimination


def generate_null_matrix(dimension):
    elementary_matrix = [[] for i in range(dimension)]
    for i in range(dimension):
        for j in range(dimension): 
            elementary_matrix[i].append(Decimal(0.0))
    return elementary_matrix


def square_matrix_multiplication(matrix_a, matrix_b):
    result_matrix = generate_null_matrix(len(matrix_a))
    for i in range(n):
        for j in range(n):
            for k in range(n):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
    return result_matrix


def ones_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        if(i < diagonal_element):
            el[i][i] = Decimal(1.0) 
        else:
            if(temporary_matrix[i][diagonal_element] == 0.0):
                el[i][i] = Decimal(1.0)
            else:
                el[i][i] = Decimal(1.0) / temporary_matrix[i][diagonal_element]
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def zeroes_column(diagonal_element, elementary_matrices_array, temporary_matrix):
    el = generate_null_matrix(len(temporary_matrix))
    for i in range(n):
        el[i][i] = Decimal(1.0)
        if(i > diagonal_element):
            if(temporary_matrix[i][diagonal_element] == 0):
                el[i][diagonal_element] = Decimal(0.0)
            else:
                el[i][diagonal_element] = Decimal(-1.0)
    elementary_matrices_array.append(el)
    temporary_matrix = square_matrix_multiplication(el, temporary_matrix)
    return elementary_matrices_array, temporary_matrix

def cut_i_row_and_jth_column(matrix, row, column):
    temp = []
    for i in range(len(matrix)):
        if(i != row):
            temp.append(matrix[i].copy())
    for i in range(len(temp)):
        temp[i].pop(column)
    return temp

def determinant(matrix):
    if(len(matrix) == 1):
        return matrix[0][0]
    det = 0
    for i in range(len(matrix)):
        if i % 2 == 0:
            det += matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
        else:
            det -= matrix[0][i] * determinant(cut_i_row_and_jth_column(matrix, 0, i))
    return det

def transpose(matrix):
    result = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        for j in range(i, len(matrix)):
            a = matrix[j][i]
            result[j][i] = matrix[i][j]
            result[i][j] = a
    return result

def inverse(matrix):
    inverse = generate_null_matrix(len(matrix))
    det = determinant(matrix)
    if det == 0:
        print("non-invertible")
        return inverse
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if (i + j) % 2 == 0: 
                inverse[i][j] = determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
            else:
                inverse[i][j] = -determinant(cut_i_row_and_jth_column(matrix, i, j)) / det
    return transpose(inverse)

def round_matrix(matrix):
    temp = []
    for i in range(len(matrix)):
        temp.append(matrix[i].copy())
        
    for i in range(len(temp)):
        for j in range(len(temp)):
            temp[i][j] = float(round(temp[i][j], 2))
    return temp

def column(matrix, j):
    result = []
    for i in range(len(matrix)):
        result.append(Decimal(matrix[i][j]))
    return result

def row(matrix, i):
    return matrix[i].copy()

def dot_product(vector1, vector2):
    result = Decimal(0)
    for i in range(len(vector1)):
        result += Decimal(vector1[i]) * Decimal(vector2[i])
    return result

def project_1_on_2(vector1, vector2):
    s = Decimal(dot_product(vector1, vector2)) / Decimal(dot_product(vector2, vector2))
    return [Decimal(i) * s for i in  vector2]

def set_column(matrix, j, vector):
    for i in range(len(matrix)):
        matrix[i][j] = Decimal(vector[i])
    return matrix

def vec1_minus_vec2(vec1, vec2):
    return [Decimal(vec1[i]) - Decimal(vec2[i]) for i in range(len(vec1))]



In [1081]:
elementary_matrices_array = []

Upper_triangular = covariance_matrix

for i in range(n):
    elementary_matrices_array, Upper_triangular = ones_column(i, elementary_matrices_array, Upper_triangular)
    elementary_matrices_array, Upper_triangular = zeroes_column(i, elementary_matrices_array, Upper_triangular)

Lower_triangular = elementary_matrices_array[-1]

for i in range(len(elementary_matrices_array) - 2, -1, -1):
    Lower_triangular = square_matrix_multiplication(Lower_triangular, elementary_matrices_array[i])
Lower_triangular = inverse(Lower_triangular)

In [1082]:
print("***Gauss LU decomposition***")
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nL matrix:")
print(round_matrix(Lower_triangular))
print("\nU matrix:")
print(round_matrix(Upper_triangular))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Lower_triangular, Upper_triangular)))


***Gauss LU decomposition***

Initial covariance matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

L matrix:
[[1.0, 0.0, 0.0, 0.0], [4.0, -5.0, 0.0, 0.0], [90.0, -270.0, 320.0, 0.0], [-100.0, 700.0, -1190.0, -167.5]]

U matrix:
[[1.0, 2.0, 3.0, 4.0], [0.0, 1.0, 2.0, 3.0], [-0.0, 0.0, 1.0, 1.25], [0.0, -0.0, 0.0, 1.0]]

Multiplied:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]


In [1083]:
#Task 2
# A = QR decomposition (or QU), Q - orhogonal, R - Upper triangular
#Firstly Gram-Schmidt:

def Gram_Schmidt_orthogonalization(matrix):
    V = generate_null_matrix(len(matrix))
    for i in range(len(matrix)):
        v_i = column(matrix, i)
        for k in range(i):
            v_i = vec1_minus_vec2(v_i, project_1_on_2(column(matrix, i), column(V, k)))
        set_column(V, i, v_i)
    return V

def Gram_Schmidt_orthonormalization(matrix):
    V = Gram_Schmidt_orthogonalization(matrix)
    for i in range(len(V)):
        set_column(V, i, [Decimal(k) / Decimal(math.sqrt(dot_product(column(V, i), column(V, i)))) for k in column(V, i) ])
    return V

def Gram_Schmidt_R(matrix, Q): 
    R = generate_null_matrix(len(matrix))
    for col in range(len(matrix)):
        for row in range(col + 1):
            R[row][col] = dot_product(column(Q, row), column(matrix, col))
    return R



In [1084]:
#QR decomposition
Q = Gram_Schmidt_orthonormalization(covariance_matrix)
R = Gram_Schmidt_R(covariance_matrix, Q)

In [1085]:
print("***QR decomposition using Gram-Schmidt***")
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nQ matrix of orthonormal basis vectors")
print(round_matrix(Q))
print("\nR matrix - upper triangular matrix")
print(round_matrix(R))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Q, R)))

***QR decomposition using Gram-Schmidt***

Initial covariance matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

Q matrix of orthonormal basis vectors
[[0.01, 0.02, 0.99, -0.14], [0.03, 0.06, 0.14, 0.99], [0.67, 0.74, -0.03, -0.06], [-0.74, 0.67, -0.01, -0.02]]

R matrix - upper triangular matrix
[[134.6, -431.55, 100.38, -66.81], [0.0, 268.1, -23.01, -6.78], [0.0, 0.0, 2.73, 5.06], [0.0, 0.0, 0.0, 2.72]]

Multiplied:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]


In [1086]:
#Task 3
#Let's find eigen values
#QR algorithm with low convergence - 100000 iterations

def eigenvalues_QR_algorithm(matrix):
    A_k = copy.deepcopy(matrix)
    U_k = generate_null_matrix(len(A_k))
    for i in range(len(U_k)):
        U_k[i][i] = Decimal(1.0)

    for k in range(100000):
        Q_k_plus1 = Gram_Schmidt_orthonormalization(A_k)
        R_k_plus1 = Gram_Schmidt_R(A_k, Q_k_plus1)
        A_k = square_matrix_multiplication(R_k_plus1, Q_k_plus1)
        U_k = square_matrix_multiplication(U_k, Q_k_plus1)
    eigenvalues = []
    for i in range(len(A_k)):
        eigenvalues.append(A_k[i][i])
    return eigenvalues, A_k, U_k


In [1087]:
eigenvalues, A, Transformation = eigenvalues_QR_algorithm(covariance_matrix)

In [1088]:
import numpy as np
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nEigenvalues from my algorithm")
print(eigenvalues)
print("\nEigenvalues from numpy")
print(np.linalg.eig(round_matrix(covariance_matrix))[0])
print("\nFinal upper triangular matrix:")
print(round_matrix(A))
print("\nAfter Transformations using QR algorithm, Gram Schmidt U*T*U_transposed which should be equal to initial")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(Transformation, A), transpose(Transformation))))


Initial covariance matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

Eigenvalues from my algorithm
[Decimal('111.6341292136725598372732579'), Decimal('-31.16613260114082677672221585'), Decimal('22.02875607154427042773730536'), Decimal('-3.496752684076011161419347693')]

Eigenvalues from numpy
[111.63412921  22.02875607  -3.49675268 -31.1661326 ]

Final upper triangular matrix:
[[111.63, -35.55, -80.83, 462.86], [0.0, -31.17, -111.45, 205.1], [-0.0, 0.0, 22.03, -30.47], [-0.0, 0.0, 0.0, -3.5]]

After Transformations using QR algorithm, Gram Schmidt U*T*U_transposed which should be equal to initial
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]


In [1089]:
#Task 4 
#We can do QR decomposition using Givens transformations

def GivensMatrix(row1, row2, column, matrix):
    G = generate_null_matrix(len(matrix))
    length = Decimal(math.sqrt(math.pow(Decimal(matrix[row1][column]), 2) + math.pow(Decimal(matrix[row2][column]), 2)))
    for i in range(len(G)):
        if(i == row1 or i == row2):
            G[i][i] = Decimal(matrix[row1][column]) / length
        else:
            G[i][i] = Decimal(1.0)
    G[row1][row2] = Decimal(matrix[row2][column]) / length
    G[row2][row1] = Decimal(-matrix[row2][column]) / length
    return G


def QR_decomposition_Givens_method(R):
    Q = generate_null_matrix(len(R))
    for i in range(len(Q)):
        Q[i][i] = Decimal(1.0)  
    for col in range(len(R)):
        for row in range(col + 1, len(R)):
            G = GivensMatrix(col, row, col, R)
            R = square_matrix_multiplication(G, R)
            Q = square_matrix_multiplication(G, Q)
    return transpose(Q), R


In [1090]:
Q, R = QR_decomposition_Givens_method(covariance_matrix)

In [1091]:
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nQ")
print(round_matrix(Q))
print("\nR")
print(round_matrix(R))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(Q,R)))



Initial covariance matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

Q
[[0.01, 0.02, 0.99, -0.14], [0.03, 0.06, 0.14, 0.99], [0.67, 0.74, -0.03, -0.06], [-0.74, 0.67, -0.01, -0.02]]

R
[[134.6, -431.55, 100.38, -66.81], [0.0, 268.1, -23.01, -6.78], [0.0, 0.0, 2.73, 5.06], [0.0, -0.0, 0.0, 2.72]]

Multiplied:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]


In [1092]:
#Task 5
#Get upper Hessenberg form

def Upper_Hessenberg(matrix):
    H = copy.deepcopy(matrix)
    PG = generate_null_matrix(len(H))
    for i in range(len(H)):
        PG[i][i] = Decimal(1.0)

    for col in range(len(matrix) - 2):
        for row in range(col + 2, len(matrix)):
            G = GivensMatrix(col + 1, row, col, H)
            H = square_matrix_multiplication(square_matrix_multiplication(G, H),transpose(G))
            PG = square_matrix_multiplication(G, PG)
    return H, PG


In [1093]:
H, PG = Upper_Hessenberg(covariance_matrix)

In [1094]:
print("\nInitial matrix:")
print(round_matrix(covariance_matrix))
print("\nUpper Hessenberg form")
print(round_matrix(H))
print("\nProduct of Givens transformations:")
print(round_matrix(PG))
print("\nMultiplied:")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(transpose(PG), H), PG)))


Initial matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

Upper Hessenberg form
[[1.0, -0.91, -4.81, 2.24], [134.6, 103.94, 300.14, 316.08], [-0.0, 3.27, 132.67, 140.57], [-0.0, 0.0, -125.93, -138.61]]

Product of Givens transformations:
[[1.0, 0.0, 0.0, 0.0], [0.0, 0.03, 0.67, -0.74], [0.0, -0.74, -0.49, -0.47], [0.0, -0.68, 0.56, 0.48]]

Multiplied:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]


In [1095]:
def QR_decomposition_Givens_for_Hessenberg(matrix):
    R = copy.deepcopy(matrix)
    Q = generate_null_matrix(len(R))
    for i in range(len(R)):
        Q[i][i] = Decimal(1.0)
    for i in range(len(R)-1):
        G = GivensMatrix(i, i+1, i, R)
        Q = square_matrix_multiplication(G, Q)
        R = square_matrix_multiplication(G, R)
    return(transpose(Q),R)
        


In [1096]:
Q,R = QR_decomposition_Givens_for_Hessenberg(H)

round_matrix(square_matrix_multiplication(Q,R))


[[1.0, -0.91, -4.81, 2.24],
 [134.6, 103.94, 300.14, 316.08],
 [-0.0, 3.27, 132.67, 140.57],
 [-0.0, 0.0, -125.93, -138.61]]

In [1097]:
#Task 6 
#Hessenberg QR Algorithm O(N^3) - 100 iterations

def HessenbergQR(matrix):
    A = copy.deepcopy(matrix)
    U_k = generate_null_matrix(len(A))
    for i in range(len(U_k)):
        U_k[i][i] = Decimal(1.0)
        
    H_k, PG = Upper_Hessenberg(A)
    
    for i in range(100):
        inverse_pg, R_k = QR_decomposition_Givens_for_Hessenberg(H_k)
        H_k = square_matrix_multiplication(R_k, inverse_pg)
        U_k = square_matrix_multiplication(U_k, inverse_pg)
    eigenvalues = []
    for i in range(len(H_k)):
        eigenvalues.append(H_k[i][i])
    U_k = square_matrix_multiplication(transpose(PG), U_k)
    return(eigenvalues, H_k, U_k)
    




In [1098]:
eigenvalues_h, H_k, Transformation_h = HessenbergQR(covariance_matrix)

In [1099]:
import numpy as np
print("\nInitial covariance matrix:")
print(round_matrix(covariance_matrix))
print("\nEigenvalues from my algorithm")
print(eigenvalues_h)
print("\nEigenvalues from numpy")
print(np.linalg.eig(round_matrix(covariance_matrix))[0])
print("\nFinal Hessenberg matrix:")
print(round_matrix(H_k))
print("\nAfter Transformations using QR algorithm, U*T*U_transposed which should be equal to initial")
print(round_matrix(square_matrix_multiplication(square_matrix_multiplication(Transformation_h, H_k), transpose(Transformation_h))))


Initial covariance matrix:
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]

Eigenvalues from my algorithm
[Decimal('111.6341292136724379175876054'), Decimal('-31.16613260114075366387995029'), Decimal('22.02875607154431284854612307'), Decimal('-3.496752684076011672644039102')]

Eigenvalues from numpy
[111.63412921  22.02875607  -3.49675268 -31.1661326 ]

Final Hessenberg matrix:
[[111.63, -35.55, 80.83, -462.86], [-0.0, -31.17, 111.45, -205.1], [-0.0, 0.0, 22.03, -30.47], [-0.0, -0.0, 0.0, -3.5]]

After Transformations using QR algorithm, U*T*U_transposed which should be equal to initial
[[1.0, 2.0, 3.0, 4.0], [4.0, 3.0, 2.0, 1.0], [90.0, -90.0, 50.0, -50.0], [-100.0, 500.0, -90.0, 45.0]]
